In [56]:
import pandas as pd
import numpy as np
import matplotlib as mplt
import keras 

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

In [57]:
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")

In [58]:
x = train_data.iloc[:,1:]
y = train_data.iloc[:,0]

In [59]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test =  train_test_split(x,y,test_size=0.1)

In [60]:
x_train = x_train.values.reshape(-1,28,28,1)
x_test = x_test.values.reshape(-1,28,28,1)

In [61]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [62]:
x_train = x_train.astype("float32")/255
x_test = x_test.astype("float32")/255
datagen.fit(x_train)

In [63]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [64]:
model = Sequential()
model.add(Conv2D(filters = 8,activation='relu',strides=1,padding = 'same', kernel_size=(5,5),data_format='channels_last',input_shape =(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 8,activation='relu',strides=1,padding='same',kernel_size=(5,5),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(3,3),padding='valid'))
model.add(Dropout(0.25))          

model.add(Conv2D(filters = 16,activation = 'relu',strides=1,padding='same',kernel_size=(5,5),data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 16,activation = 'relu',strides=1,padding='same',kernel_size=(5,5),data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(strides=2,pool_size=(3,3),padding='valid'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

In [65]:
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999 )

In [66]:
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [67]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [68]:
batch_size = 128
epochs = 40
history = model.fit(datagen.flow(x_train,y_train,batch_size=batch_size),epochs=epochs,
validation_data=(x_test,y_test),verbose=1,steps_per_epoch=x_train.shape[0]//batch_size,callbacks=[reduce_lr])

Epoch 1/40
295/295 [==============================] - 14s 39ms/step - loss: 0.5899 - accuracy: 0.8279 - val_loss: 3.5775 - val_accuracy: 0.2890 - lr: 0.0010
Epoch 2/40
295/295 [==============================] - 12s 41ms/step - loss: 0.1850 - accuracy: 0.9436 - val_loss: 0.0468 - val_accuracy: 0.9833 - lr: 9.0000e-04
Epoch 3/40
295/295 [==============================] - 12s 40ms/step - loss: 0.1375 - accuracy: 0.9569 - val_loss: 0.0433 - val_accuracy: 0.9864 - lr: 8.1000e-04
Epoch 4/40
295/295 [==============================] - 12s 39ms/step - loss: 0.1091 - accuracy: 0.9663 - val_loss: 0.0322 - val_accuracy: 0.9890 - lr: 7.2900e-04
Epoch 5/40
295/295 [==============================] - 12s 39ms/step - loss: 0.0915 - accuracy: 0.9725 - val_loss: 0.0318 - val_accuracy: 0.9895 - lr: 6.5610e-04
Epoch 6/40
295/295 [==============================] - 12s 39ms/step - loss: 0.0811 - accuracy: 0.9745 - val_loss: 0.0334 - val_accuracy: 0.9881 - lr: 5.9049e-04
Epoch 7/40
295/295 [==================